In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session




# # confirm TensorFlow sees the GPU
# from tensorflow.python.client import device_lib
# assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU (for TensorFlow 1.X + Keras)
# from keras import backend
# assert len(backend.tensorflow_backend._get_available_gpus()) > 0


# from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future
import tensorflow as tf
# import tensorflow.compat.v1 as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras import backend
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt


from glob import glob

In [ ]:
# make sure GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))

In [ ]:
# label dataframe inspection
train_labels = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv")
train_labels.head()

In [ ]:
# define train/test path
train_path = '../input/histopathologic-cancer-detection/train'
valid_path = '../input/histopathologic-cancer-detection/test'

# useful for getting number of files
image_files = glob(train_path + '/*.tif')
valid_image_files = glob(valid_path + '/*.tif')

# image snapshot
for i in range(5):
    _label = 0
    image_path = np.random.choice(train_labels[train_labels['label'] == _label]['id'].apply(lambda x: train_path + "/" + x + ".tif"))
    #print(image_path)
    plt.title(f"Path: {image_path} | label : {_label}")
    plt.imshow(image.load_img(image_path))
    
    plt.show()
    
for i in range(5):
    _label = 1
    image_path = np.random.choice(train_labels[train_labels['label'] == _label]['id'].apply(lambda x: train_path + "/" + x + ".tif"))
    #print(image_path)
    plt.title(f"Path: {image_path} | label : {_label}")
    plt.imshow(image.load_img(image_path))
    
    plt.show()


In [ ]:
# img.shape

# inspect csv 

In [ ]:
# !mkdir ./train_set
# !mkdir ./train_set/label_0
# !mkdir ./train_set/label_1

In [ ]:
# !ls train_set/


In [ ]:
# train_path = '../input/histopathologic-cancer-detection/train'
# valid_path = '../input/histopathologic-cancer-detection/test'

In [ ]:
# train_set_label = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv")
# train_set_label[train_set_label['label'] == 1].head(10)

In [ ]:

# import shutil

# for index, row in train_set_label[train_set_label['label'] == 1].iterrows():
#     img_path = f"../input/histopathologic-cancer-detection/train/{row['id']}.tif"
#     print(index, img_path)
#     shutil.copy2(img_path, f"/kaggle/working/label_1") # complete target filename given

# for index, row in train_set_label[train_set_label['label'] == 0].iterrows():
#     img_path = f"../input/histopathologic-cancer-detection/train/{row['id']}.tif"
#     print(index, img_path)
#     shutil.copy2(img_path, f"/kaggle/working/label_0") # complete target filename given


In [ ]:
# image_files = glob(train_path + '/*tif')
# len(image_files)
# # print(train_path)

In [ ]:
# !ls ./train_set/label_1/
# !pwd 

# Use pretrained model

In [ ]:
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
tf.compat.v1.Session()
sess = tf.compat.v1.Session(config=config)
# sess = tf.Session(config=config) 
# backend.set_session(sess)
sess = tf.compat.v1.keras.backend.get_session()

In [ ]:
#print hardware spec.
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
!pip install keras==2.1 -i https://pypi.douban.com/simple/

In [ ]:
# from keras import backend as K
# # tf.compat.v1.keras.backend.
# # K.tensorflow_backend._get_available_gpus()
# # K.tensorflow_backend._get_available_gpus()

# dir(K)


In [ ]:


train_labels = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv")
train_labels['id'] = train_labels['id'].apply(lambda x: '{}.tif'.format(x))

# Data augmentation pipeline
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
# Reading files from path in data frame
train_ds = train_datagen.flow_from_dataframe(train_labels,directory = '../input/histopathologic-cancer-detection/train', x_col = 'id', y_col = 'label', class_mode='raw')

In [ ]:
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50, preprocess_input
# from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
import numpy as np


from glob import glob


# re-size all the images to this
IMAGE_SIZE = [96, 96] # 

# training config:
epochs = 16
batch_size = 32

# # https://www.kaggle.com/paultimothymooney/blood-cells
# train_path = '../input/histopathologic-cancer-detection/train'
# valid_path = '../input/histopathologic-cancer-detection/test'

# # https://www.kaggle.com/moltean/fruits
# # train_path = '../large_files/fruits-360/Training'
# # valid_path = '../large_files/fruits-360/Validation'
# # train_path = '../large_files/fruits-360-small/Training'
# # valid_path = '../large_files/fruits-360-small/Validation'

# # useful for getting number of files
# image_files = glob(train_path + '/*.tif')
# valid_image_files = glob(valid_path + '/*.tif')

# # useful for getting number of classes
# # folders = glob(train_path + '/*')


# # # look at an image 
# # for i in range(3):
# #     plt.imshow(image.load_img(np.random.choice(image_files)))
# #     plt.show()

# # add preprocessing layer to the front of VGG











In [ ]:
res = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in res.layers:
    layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(res.output)
x = Dense(1000, activation='relu')(x)
#prediction = Dense(2, activation='softmax')(x)
# prediction = Dense(2, activation='sigmoid')(x)
prediction = Dense(1, activation='sigmoid')(x)
# model.add(layers.Dense(1, activation='sigmoid'))
# create a model object
model = Model(inputs=res.input, outputs=prediction)


# tell the model what cost and optimization method to use
model.compile(
  #loss='categorical_crossentropy',
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)



# image-label mapping
df = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv")
df['id'] = df['id'].apply(lambda x: '{}.tif'.format(x))

# Data augmentation pipeline
# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
# create an instance of ImageDataGenerator
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
    validation_split=0.2
)
# Reading files from path in data frame
train_generator = train_datagen.flow_from_dataframe(
        dataframe = df,
        directory = '../input/histopathologic-cancer-detection/train', 
        x_col = 'id',
        y_col = 'label', 
        class_mode='raw',
        target_size=IMAGE_SIZE,
        subset='training' # set as training data
)

validation_generator = train_datagen.flow_from_dataframe(
        dataframe = df,
        directory = '../input/histopathologic-cancer-detection/train', 
        x_col = 'id',
        y_col = 'label', 
        class_mode='raw',
        target_size=IMAGE_SIZE,
        subset='validation' # set as validation data
)

# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     class_mode='binary',
#     subset='training') # set as training data

# validation_generator = train_datagen.flow_from_directory(
#     train_data_dir, # same directory as training data
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     class_mode='binary',
#     subset='validation') # set as validation data

# fit the model
r = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = epochs
)

In [ ]:
# view the structure of the model
model.summary()

In [ ]:
# Plot loss per iteration
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()

In [ ]:
# save trained model
model.save_weights('roger_trained_model_histopathologic_cancer_detection_checkpoint.h5')
# ../input/histopathologic-cancer-detection

In [ ]:
!ls

In [ ]:
train_labels['id'] = train_labels['id'].apply(lambda x: '{}.tif'.format(x))

# Prediction (zero matching rate)   ><"  sosad !!

In [ ]:
#for date, row in train_labels[train_labels['label'] == 1].head(10000).iterrows():
for date, row in train_labels.head(100).iterrows():
    #print(f"{row['id']} | label: {row['label']}")
    
    image_path = f"../input/histopathologic-cancer-detection/train/{row['id']}"
    #print(image_path)
    

    input_arr = tf.keras.preprocessing.image.img_to_array(image.load_img(image_path, target_size=[96,96],color_mode='rgb' ))
#     input_arr = cv2.imread(image_path)
    input_arr2 = np.array([input_arr])
    input_arr3 = input_arr2.astype('float32') / 255
    
    predictions = model.predict(input_arr3)
    predicted = np.argmax(predictions, axis=-1)[0]
    
    print(f"{row['id']} | label: {row['label']} | predicted: {predicted}")